In [1]:
from pathlib import Path
import os
import numpy as np
import torch
import torchvision.transforms.functional as F
from matplotlib import pyplot as plt
from torchvision.utils import draw_segmentation_masks, make_grid
import pytorch_lightning as pl
import mlflow
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.callbacks import RichProgressBar, ModelCheckpoint

In [2]:
from inz.data.event import Event, Hold, Test, Tier1, Tier3
from inz.data.data_module import XBDDataModule
from inz.models.unet_basic import UNet
from inz.models.unet_basic_pl import SemanticSegmentor, OrdinalCrossEntropyLoss

In [3]:
RANDOM_SEED = 123
pl.seed_everything(RANDOM_SEED)
device = torch.device("cuda")
torch.set_float32_matmul_precision("high")

Seed set to 123


In [4]:
dm = XBDDataModule(
    path=Path("data/xBD_processed"),
    events={
        Tier1: [
            Event.hurricane_florence,
            Event.hurricane_harvey,
            Event.hurricane_matthew,
            Event.hurricane_michael,
        ],
        Tier3: [
            Event.joplin_tornado,
            Event.moore_tornado,
            Event.tuscaloosa_tornado
        ],
        Hold: [
            Event.hurricane_florence,
            Event.hurricane_harvey,
            Event.hurricane_matthew,
            Event.hurricane_michael,
        ],
        Test: [
            Event.hurricane_florence,
            Event.hurricane_harvey,
            Event.hurricane_matthew,
            Event.hurricane_michael,
        ],
    },
    val_faction=0.1,
    test_fraction=0.1,
    train_batch_size=4,
    val_batch_size=4,
    test_batch_size=4
)
dm.prepare_data()
dm.setup("fit")

print(f"{len(dm.train_dataloader())} train examples, {len(dm.val_dataloader())} val examples, {len(dm.test_dataloader())} test examples, ")

2194 train examples, 274 val examples, 274 test examples, 


In [5]:


model = SemanticSegmentor(
    model=UNet(in_channels=3, out_channels=5),
    localization_loss=OrdinalCrossEntropyLoss(n_classes=5),
    classification_loss=OrdinalCrossEntropyLoss(n_classes=5),
    n_classes=5
)

/home/tomek/inz/inz/.venv/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
/home/tomek/inz/inz/.venv/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'localization_loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['localization_loss'])`.
/home/tomek/inz/inz/.venv/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'classification_loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['classification_loss'])`.


In [6]:
mlflow.pytorch.autolog()


checkpoint_callback = ModelCheckpoint(
    save_top_k=1, verbose=True, monitor="val_loss", mode="min"
)

trainer = pl.Trainer(
    max_epochs=30,
    callbacks=[
        RichProgressBar(),
        # checkpoint_callback
    ],
    logger=MLFlowLogger(),
    precision=16
)
trainer.fit(model, datamodule=dm)

/home/tomek/inz/inz/.venv/lib/python3.11/site-packages/lightning_fabric/connector.py:563: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024/06/09 00:07:46 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '821565fce8c94d518468eedee40bd06f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                ┃ Type                    ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model               │ UNet                    │  124 M │
│ 1 │ localization_loss   │ OrdinalCrossEntropyLoss │      0 │
│ 2 │ classification_loss │ OrdinalCrossEntropyLoss │      0 │
│ 3 │ f1                  │ MulticlassF1Score       │      0 │
│ 4 │ precision           │ MulticlassPrecision     │      0 │
│ 5 │ recall              │ MulticlassRecall        │      0 │
│ 6 │ iou                 │ MeanIoU                 │      0 │
└───┴─────────────────────┴─────────────────────────┴────────┘

Trainable params: 124 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 124 M                                                                                                
Total estimated model params size (MB): 496

Output()

/home/tomek/inz/inz/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected 
KeyboardInterrupt, attempting graceful shutdown...